In [1]:
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
import pandas as pd
from nltk.corpus import stopwords, wordnet
import string
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("/Users/vedantsingh/Downloads/twitter-airline-sentiment/Tweets.csv")
air = ['southwestair', 'jetblue', 'usairways', 'united', 'virginamerica', 'americanair']

In [3]:
data.drop(columns = ['tweet_id', 'airline', 'airline_sentiment_gold', 'name', 'negativereason_gold', 'retweet_count', 'tweet_coord', 'tweet_created', 'tweet_location', 'user_timezone', 'airline_sentiment_confidence', 'negativereason', 'negativereason_confidence'], inplace = True)

In [4]:
docs = []
for i in range(len(data)) :
    docs.append((word_tokenize(data.iloc[i, 1]), data.iloc[i, 0]))

In [5]:
stops = list(stopwords.words('english'))
puncs = list(string.punctuation)
stops = stops + puncs + air

In [6]:
def simple_pos(pos) :
    if pos.startswith('J') :
        return wordnet.ADJ
    elif pos.startswith('N') :
        return wordnet.NOUN
    elif pos.startswith('V') :
        return wordnet.VERB
    elif pos.startswith('R') :
        return wordnet.ADV
    else :
        return wordnet.NOUN

In [7]:
lemmatizer = WordNetLemmatizer()
clean_docs = []
for i in range (len(docs)) :
    output = []
    for w in docs[i][0] :
        if w.lower() not in stops :
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = simple_pos(pos[0][1]))
            output.append(clean_word.lower())
    clean_docs.append((output, docs[i][1]))

In [8]:
train, test = train_test_split(clean_docs)

In [9]:
train_docs = [" ".join(docs) for docs, cat in train]
count_vec = CountVectorizer(max_features = 4000)
a = count_vec.fit_transform(train_docs)
x_train = a.todense()
y_train = [cat for doc, cat in train]

In [10]:
test_docs = [" ".join(docs) for docs, cat in test]
b = count_vec.transform(test_docs)
x_test = b.todense()
y_test = [cat for doc, cat in test]

In [11]:
bnb = BernoulliNB()
bnb.fit(x_train, y_train)
bnb.score(x_train, y_train), bnb.score(x_test, y_test)

(0.8332422586520947, 0.7644808743169399)